In [1]:
# get db url 
#create db engine via sqlalchemy    

In [1]:
from env import config, BASE_DIR
from sqlalchemy import create_engine, text

In [2]:
DATABASE_URL = config("HOUSING_DATABASE_URL", cast=str, default=None)
DATABASE_URL is not None

True

In [3]:
engine = create_engine(str(DATABASE_URL))

In [4]:
DATASET_DIR = BASE_DIR / "datasets"
DATASET_DIR.exists()

True

In [5]:
HOUSES_CSV = DATASET_DIR / "housing_ds.csv"
HOUSES_CSV.exists()

True

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv(str(HOUSES_CSV))
df.head(n=10)

,datesold,postcode,price,propertyType,bedrooms
0,2007-02-07 00:00:00,2607,525000,house,4
1,2007-02-27 00:00:00,2906,290000,house,3
2,2007-03-07 00:00:00,2905,328000,house,3
3,2007-03-09 00:00:00,2905,380000,house,4
4,2007-03-21 00:00:00,2906,310000,house,3
5,2007-04-04 00:00:00,2905,465000,house,4
6,2007-04-24 00:00:00,2607,399000,house,3
7,2007-04-30 00:00:00,2606,1530000,house,4
8,2007-05-24 00:00:00,2902,359000,house,3
9,2007-05-25 00:00:00,2906,320000,house,3


In [8]:
df.dtypes


datesold        object
postcode         int64
price            int64
propertyType    object
bedrooms         int64
dtype: object

In [9]:
df['date'] = pd.to_datetime(df['datesold'], format="%Y-%m-%d %H:%M:%S")

In [10]:
df.dtypes


datesold                object
postcode                 int64
price                    int64
propertyType            object
bedrooms                 int64
date            datetime64[ns]
dtype: object

In [11]:
df.head()

,datesold,postcode,price,propertyType,bedrooms,date
0,2007-02-07 00:00:00,2607,525000,house,4,2007-02-07
1,2007-02-27 00:00:00,2906,290000,house,3,2007-02-27
2,2007-03-07 00:00:00,2905,328000,house,3,2007-03-07
3,2007-03-09 00:00:00,2905,380000,house,4,2007-03-09
4,2007-03-21 00:00:00,2906,310000,house,3,2007-03-21


In [12]:
out_df = df.copy()[['date','price','propertyType','bedrooms', 'postcode']]

In [13]:
table_name = "kaggle_house_data"
out_df.to_sql(table_name, engine, if_exists='replace', index=False)

29580

In [14]:
df.shape

(29580, 6)

In [15]:
with engine.connect() as conn:
    results = conn.execute(text(f"SELECT * from {table_name} LIMIT 10"))
    print(results.all())

[(datetime.datetime(2007, 2, 7, 0, 0), 525000, 'house', 4, 2607), (datetime.datetime(2007, 2, 27, 0, 0), 290000, 'house', 3, 2906), (datetime.datetime(2007, 3, 7, 0, 0), 328000, 'house', 3, 2905), (datetime.datetime(2007, 3, 9, 0, 0), 380000, 'house', 4, 2905), (datetime.datetime(2007, 3, 21, 0, 0), 310000, 'house', 3, 2906), (datetime.datetime(2007, 4, 4, 0, 0), 465000, 'house', 4, 2905), (datetime.datetime(2007, 4, 24, 0, 0), 399000, 'house', 3, 2607), (datetime.datetime(2007, 4, 30, 0, 0), 1530000, 'house', 4, 2606), (datetime.datetime(2007, 5, 24, 0, 0), 359000, 'house', 3, 2902), (datetime.datetime(2007, 5, 25, 0, 0), 320000, 'house', 3, 2906)]
